In [1]:
try:
  import requests
  from bs4 import BeautifulSoup
  import pandas as pd
except:
  !pip3 install requests
  !pip3 install beautifulsoup4
  !pip3 install pandas
  import requests
  from bs4 import BeautifulSoup
  import pandas as pd

# Match-By-Match Batting Stats Scraping

In [ ]:
# Define the base URL and the initial page URL
base_url = 'https://stats.espncricinfo.com'
url = base_url + '/ci/engine/stats/index.html?class=3;page=1;spanmin1=1+Jan+2022;spanval1=span;template=results;type=batting;view=innings'

response = requests.get(url)
response = response.content
soup = BeautifulSoup(response, 'html.parser')

In [ ]:
# Initialize the page number
page_number = 1
total_pages = int(soup.find_all('td', class_='left')[3].text.split(' ')[6].rstrip())

In [ ]:
page_number

1

In [ ]:
total_pages

335

In [ ]:
data = []
for i in range(1, total_pages+1):
  url = f"https://stats.espncricinfo.com/ci/engine/stats/index.html?class=3;page={i};spanmin1=1+Jan+2022;spanval1=span;template=results;type=batting;view=innings"
  response = requests.get(url)
  response = response.content
  soup = BeautifulSoup(response, 'html.parser')
  # Find the table element that contains the data we want to extract
  table = soup.select_one('#ciHomeContentlhs > div.pnl650M > table:nth-child(5)')

  # Check if the table exists
  if table is not None:
    # Loop through the rows of the table and extract the data for each player
    for row in table.tbody.find_all('tr'):
      columns = row.find_all('td')
      if len(columns) > 0:
        Player = columns[0].text.strip()
        Runs = columns[1].text.strip()
        Mins = columns[2].text.strip()
        BF = columns[3].text.strip()
        Fours = columns[4].text.strip()
        Sixes = columns[5].text.strip()
        SR = columns[6].text.strip()
        Inns = columns[7].text.strip()
        Opposition = columns[9].text.strip()
        Ground = columns[10].text.strip()
        Start_Date = columns[11].text.strip()

        # Check if the Runs column has a '*'
        if '*' in Runs:
          Not_Out = 1
        else:
          Not_Out = 0

        # Trim the asterisk (*) from the Runs column
        Runs = Runs.strip('*')

        # Add the data for each player to the DataFrame
        data.append({
          'Player': Player,
          'Runs': Runs,
          'Mins': Mins,
          'Not_Out': Not_Out,
          'BF': BF,
          'Fours': Fours,
          'Sixes': Sixes,
          'SR': SR,
          'Inns': Inns,
          'Opposition': Opposition,
          'Ground': Ground,
          'Start_Date': Start_Date
        })

    # Check if there is a next page by finding the 'Next' button on the page
    next_page_link = soup.select_one('.PaginationLink')
    if next_page_link is not None:
      # If there is a next page, increment the page number and update the URL to the next page
      i += 1
    else:
      # If there is no next page, break out of the loop
      break

# Create a DataFrame from the scraped data
df = pd.DataFrame(data)

In [ ]:
# Print the DataFrame
df.head(10)

,Player,Runs,Mins,Not_Out,BF,Fours,Sixes,SR,Inns,Opposition,Ground,Start_Date
0,Zeeshan Kukikhel (HUN),137,-,0,49,7,15,279.59,2,v Austria,Lower Austria,5 Jun 2022
1,HG Munsey (SCOT),132,-,0,61,15,6,216.39,1,v Austria,Edinburgh (Golden),25 Jul 2023
2,OJ Hairs (SCOT),127,-,1,53,14,8,239.62,1,v Italy,Edinburgh,24 Jul 2023
3,Shubman Gill (IND),126,99,1,63,12,7,200.00,1,v New Zealand,Ahmedabad,1 Feb 2023
4,V Kohli (IND),122,90,1,61,12,6,200.00,1,v Afghanistan,Dubai (DSC),8 Sep 2022
5,J Charles (WI),118,60,0,46,10,11,256.52,1,v South Africa,Centurion,26 Mar 2023
6,LA Dunbar (SRB),117,-,0,50,9,11,234.00,1,v Bulgaria,Sofia,26 Jun 2022
7,SA Yadav (IND),117,88,0,55,14,6,212.72,2,v England,Nottingham,10 Jul 2022
8,Virandeep Singh (MAL),116,-,1,62,11,7,187.09,1,v Indonesia,Phnom Penh,2 May 2023
9,S Davizi (CZK-R),115,-,1,59,7,8,194.91,1,v Bulgaria,Marsa,12 May 2022


In [ ]:
# Trim the v before the country name in the opposition column
df['Opposition'] = df['Opposition'].str.lstrip('v ')

In [ ]:
# Extract player name and country using regular expression
df[['Player_Name', 'Country']] = df['Player'].str.extract(r'^(.*?) \((.*?)\)')

In [ ]:
df

,Player,Runs,Mins,Not_Out,BF,Fours,Sixes,SR,Inns,Opposition,Ground,Start_Date,Player_Name,Country
0,Zeeshan Kukikhel (HUN),137,-,0,49,7,15,279.59,2,Austria,Lower Austria,5 Jun 2022,Zeeshan Kukikhel,HUN
1,HG Munsey (SCOT),132,-,0,61,15,6,216.39,1,Austria,Edinburgh (Golden),25 Jul 2023,HG Munsey,SCOT
2,OJ Hairs (SCOT),127,-,1,53,14,8,239.62,1,Italy,Edinburgh,24 Jul 2023,OJ Hairs,SCOT
3,Shubman Gill (IND),126,99,1,63,12,7,200.00,1,New Zealand,Ahmedabad,1 Feb 2023,Shubman Gill,IND
4,V Kohli (IND),122,90,1,61,12,6,200.00,1,Afghanistan,Dubai (DSC),8 Sep 2022,V Kohli,IND
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16740,Shaik Basha (TAN),DNB,-,0,-,-,-,-,2,Rwanda,Rwanda,22 Aug 2023,Shaik Basha,TAN
16741,SA Jumbe (TAN),DNB,-,0,-,-,-,-,2,Rwanda,Rwanda,22 Aug 2023,SA Jumbe,TAN
16742,YM Nkanya (TAN),DNB,-,0,-,-,-,-,2,Rwanda,Rwanda,22 Aug 2023,YM Nkanya,TAN
16743,AM Kimote (TAN),DNB,-,0,-,-,-,-,2,Rwanda,Rwanda,22 Aug 2023,AM Kimote,TAN


In [ ]:
# Reorder columns
df = df[['Player_Name', 'Country', 'Runs', 'Not_Out', 'BF', 'Fours', 'Sixes', 'SR', 'Inns', 'Opposition', 'Ground', 'Start_Date']]

In [ ]:
def replace_values(value):
    if value == 'DNB':
      return 0
    elif value == 'TDNB':
      return 0
    elif value == 'absent':
      return 0
    elif value == 'sub':
      return 0
    elif value == '-':
      return 0
    else:
      return value

# apply the replace_values function to all columns in the DataFrame
df = df.applymap(replace_values)
df

In [ ]:
df.to_excel('batters.xlsx')

# Match-By-Match Bowling Stats Scraping

In [ ]:
# Define the base URL and the initial page URL
base_url = 'https://stats.espncricinfo.com'
url = base_url + '/ci/engine/stats/index.html?class=3;page=1;spanmin1=1+Jan+2022;spanval1=span;template=results;type=bowling;view=innings'

response = requests.get(url)
response = response.content
soup = BeautifulSoup(response, 'html.parser')

In [ ]:
# Initialize the page number
page_number = 1
total_pages = int(soup.find_all('td', class_='left')[3].text.split(' ')[6].rstrip())

In [ ]:
data2 = []
for i in range(1, total_pages+1):
  url = f"https://stats.espncricinfo.com/ci/engine/stats/index.html?class=3;page={i};spanmin1=1+Jan+2022;spanval1=span;template=results;type=bowling;view=innings"
  response = requests.get(url)
  response = response.content
  soup = BeautifulSoup(response, 'html.parser')
  # Find the table element that contains the data we want to extract
  table = soup.select_one('#ciHomeContentlhs > div.pnl650M > table:nth-child(5)')

  # Check if the table exists
  if table is not None:
    # Loop through the rows of the table and extract the data for each player
    for row in table.tbody.find_all('tr'):
      columns = row.find_all('td')
      if len(columns) > 0:
        Player = columns[0].text.strip()
        Overs = columns[1].text.strip()
        Maidens = columns[2].text.strip()
        Runs = columns[3].text.strip()
        Wickets = columns[4].text.strip()
        Economy = columns[5].text.strip()
        Inns = columns[6].text.strip()
        Opposition = columns[8].text.strip()
        Ground = columns[9].text.strip()
        Start_Date = columns[10].text.strip()

        # Add the data for each player to the DataFrame
        data2.append({
          'Player': Player,
          'Overs': Overs,
          'Maidens': Maidens,
          'Runs': Runs,
          'Wickets': Wickets,
          'Economy': Economy,
          'Inns': Inns,
          'Opposition': Opposition,
          'Ground': Ground,
          'Start_Date': Start_Date
        })

    # Check if there is a next page by finding the 'Next' button on the page
    next_page_link = soup.select_one('.PaginationLink')
    if next_page_link is not None:
      # If there is a next page, increment the page number and update the URL to the next page
      i += 1
    else:
      # If there is no next page, break out of the loop
      break

# Create a DataFrame from the scraped data
df2 = pd.DataFrame(data2)

In [ ]:
# Print the DataFrame
df2.head(10)

In [ ]:
# Trim the v before the country name in the opposition column
df2['Opposition'] = df2['Opposition'].str.lstrip('v ')

In [ ]:
# Extract player name and country using regular expression
df2[['Player_Name', 'Country']] = df2['Player'].str.extract(r'^(.*?) \((.*?)\)')

In [ ]:
df2

In [ ]:
# Reorder columns
df2 = df2[['Player_Name', 'Country', 'Overs', 'Maidens', 'Runs', 'Wickets', 'Economy', 'Inns', 'Opposition', 'Ground', 'Start_Date']]

In [ ]:
df2

In [ ]:
df2.to_excel('bowlers.xlsx')

# Overall Batting Stats

In [ ]:
# Define the initial page URL
url = 'https://stats.espncricinfo.com/ci/engine/stats/index.html?class=3;filter=advanced;orderby=runs;page=1;size=200;spanmin1=1+Jan+2022;spanval1=span;template=results;type=batting'

response = requests.get(url)
response = response.content
soup = BeautifulSoup(response, 'html.parser')

In [ ]:
# Initialize the page number
page_number = 1
total_pages = int(soup.find_all('td', class_='left')[3].text.split(' ')[6].rstrip())

In [ ]:
page_number

1

In [ ]:
total_pages

10

In [ ]:
data = []
for i in range(1, total_pages+1):
  url = f"https://stats.espncricinfo.com/ci/engine/stats/index.html?class=3;filter=advanced;orderby=runs;page={i};size=200;spanmin1=1+Jan+2022;spanval1=span;template=results;type=batting"
  response = requests.get(url)
  response = response.content
  soup = BeautifulSoup(response, 'html.parser')
  # Find the table element that contains the data we want to extract
  table = soup.select_one('#ciHomeContentlhs > div.pnl650M > table:nth-child(5)')

  # Check if the table exists
  if table is not None:
    # Loop through the rows of the table and extract the data for each player
    for row in table.tbody.find_all('tr'):
      columns = row.find_all('td')
      if len(columns) > 0:
        Player = columns[0].text.strip()
        Span = columns[1].text.strip()
        Matches = columns[2].text.strip()
        Innings = columns[3].text.strip()
        Not_Out = columns[4].text.strip()
        Runs = columns[5].text.strip()
        Highest_Score = columns[6].text.strip()
        Average = columns[7].text.strip()
        Balls_Faced = columns[8].text.strip()
        SR = columns[9].text.strip()
        Hundreds = columns[10].text.strip()
        Fifties = columns[11].text.strip()
        Zeros = columns[12].text.strip()
        Fours = columns[13].text.strip()
        Sixes = columns[14].text.strip()


        # Add the data for each player to the DataFrame
        data.append({
          'Player': Player,
          'Span': Span,
          'Matches': Matches,
          'Innings': Innings,
          'Not_Out' : Not_Out,
          'Runs': Runs,
          'Highest_Score': Highest_Score,
          'Average' : Average,
          'Balls_Faced': Balls_Faced,
          'SR': SR,
          'Hundreds': Hundreds,
          'Fifties': Fifties,
          'Zeros' : Zeros,
          'Fours': Fours,
          'Sixes': Sixes
        })

    # Check if there is a next page by finding the 'Next' button on the page
    next_page_link = soup.select_one('.PaginationLink')
    if next_page_link is not None:
      # If there is a next page, increment the page number and update the URL to the next page
      i += 1
    else:
      # If there is no next page, break out of the loop
      break

# Create a DataFrame from the scraped data
df3 = pd.DataFrame(data)

In [ ]:
# Print the DataFrame
df3.head(10)

,Player,Span,Matches,Innings,Not_Out,Runs,Highest_Score,Average,Balls_Faced,SR,Hundreds,Fifties,Zeros,Fours,Sixes
0,SA Yadav (IND),2022-2023,42,41,8,1597,117,48.39,909,175.68,3,12,2,141,92
1,Mohammad Rizwan (PAK),2022-2023,30,29,4,1158,98*,46.32,926,125.05,0,12,0,93,27
2,S Ssesazi (UGA),2022-2023,41,39,4,1043,100*,29.80,817,127.66,1,8,7,110,35
3,Virandeep Singh (MAL),2022-2023,32,31,7,999,116*,41.62,719,138.94,1,7,3,95,45
4,Babar Azam (PAK),2022-2023,31,30,4,865,110*,33.26,685,126.27,2,5,2,99,13
5,Muhammad Waseem (UAE),2022-2023,22,22,0,858,112,39.00,583,147.16,1,8,1,59,57
6,A Balbirnie (IRE),2022-2023,36,36,1,848,75*,24.22,668,126.94,0,6,2,78,41
7,Litton Das (BAN),2022-2023,27,27,0,826,83,30.59,587,140.71,0,6,0,93,23
8,II Selemani (TAN),2022-2023,35,34,3,821,71,26.48,501,163.87,0,4,4,91,48
9,N Pooran (WI),2022-2023,31,30,2,817,74*,29.17,599,136.39,0,6,0,57,47


In [ ]:
df3 = df3.drop('Span', axis=1)
df3

,Player,Matches,Innings,Not_Out,Runs,Highest_Score,Average,Balls_Faced,SR,Hundreds,Fifties,Zeros,Fours,Sixes
0,SA Yadav (IND),42,41,8,1597,117,48.39,909,175.68,3,12,2,141,92
1,Mohammad Rizwan (PAK),30,29,4,1158,98*,46.32,926,125.05,0,12,0,93,27
2,S Ssesazi (UGA),41,39,4,1043,100*,29.80,817,127.66,1,8,7,110,35
3,Virandeep Singh (MAL),32,31,7,999,116*,41.62,719,138.94,1,7,3,95,45
4,Babar Azam (PAK),31,30,4,865,110*,33.26,685,126.27,2,5,2,99,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1915,Yash Talati (KENYA),2,-,-,-,-,-,-,-,-,-,-,-,-
1916,Yousuf Ali (QAT),1,-,-,-,-,-,-,-,-,-,-,-,-
1917,Zahir Khan (AFG),1,-,-,-,-,-,-,-,-,-,-,-,-
1918,Zeeshan Abbas (BHR),3,-,-,-,-,-,-,-,-,-,-,-,-


In [ ]:
def replace_values(value):
    if value == 'DNB':
      return 0
    elif value == 'TDNB':
      return 0
    elif value == 'absent':
      return 0
    elif value == 'sub':
      return 0
    elif value == '-':
      return 0
    else:
      return value

# apply the replace_values function to all columns in the DataFrame
df3 = df3.applymap(replace_values)
df3

,Player,Matches,Innings,Not_Out,Runs,Highest_Score,Average,Balls_Faced,SR,Hundreds,Fifties,Zeros,Fours,Sixes
0,SA Yadav (IND),42,41,8,1597,117,48.39,909,175.68,3,12,2,141,92
1,Mohammad Rizwan (PAK),30,29,4,1158,98*,46.32,926,125.05,0,12,0,93,27
2,S Ssesazi (UGA),41,39,4,1043,100*,29.80,817,127.66,1,8,7,110,35
3,Virandeep Singh (MAL),32,31,7,999,116*,41.62,719,138.94,1,7,3,95,45
4,Babar Azam (PAK),31,30,4,865,110*,33.26,685,126.27,2,5,2,99,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1915,Yash Talati (KENYA),2,0,0,0,0,0,0,0,0,0,0,0,0
1916,Yousuf Ali (QAT),1,0,0,0,0,0,0,0,0,0,0,0,0
1917,Zahir Khan (AFG),1,0,0,0,0,0,0,0,0,0,0,0,0
1918,Zeeshan Abbas (BHR),3,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
# Extract player name and country using regular expression
df3[['Player_Name', 'Country']] = df3['Player'].str.extract(r'^(.*?) \((.*?)\)')

In [ ]:
df3

,Player,Matches,Innings,Not_Out,Runs,Highest_Score,Average,Balls_Faced,SR,Hundreds,Fifties,Zeros,Fours,Sixes,Player_Name,Country
0,SA Yadav (IND),42,41,8,1597,117,48.39,909,175.68,3,12,2,141,92,SA Yadav,IND
1,Mohammad Rizwan (PAK),30,29,4,1158,98*,46.32,926,125.05,0,12,0,93,27,Mohammad Rizwan,PAK
2,S Ssesazi (UGA),41,39,4,1043,100*,29.80,817,127.66,1,8,7,110,35,S Ssesazi,UGA
3,Virandeep Singh (MAL),32,31,7,999,116*,41.62,719,138.94,1,7,3,95,45,Virandeep Singh,MAL
4,Babar Azam (PAK),31,30,4,865,110*,33.26,685,126.27,2,5,2,99,13,Babar Azam,PAK
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1915,Yash Talati (KENYA),2,0,0,0,0,0,0,0,0,0,0,0,0,Yash Talati,KENYA
1916,Yousuf Ali (QAT),1,0,0,0,0,0,0,0,0,0,0,0,0,Yousuf Ali,QAT
1917,Zahir Khan (AFG),1,0,0,0,0,0,0,0,0,0,0,0,0,Zahir Khan,AFG
1918,Zeeshan Abbas (BHR),3,0,0,0,0,0,0,0,0,0,0,0,0,Zeeshan Abbas,BHR


In [ ]:
df3.to_excel('Overall_batters.xlsx')

# Overall Bowling Stats

In [ ]:
# Define the initial page URL
url = 'https://stats.espncricinfo.com/ci/engine/stats/index.html?class=3;filter=advanced;orderby=wickets;page=1;size=200;spanmin1=1+Jan+2022;spanval1=span;template=results;type=bowling'

response = requests.get(url)
response = response.content
soup = BeautifulSoup(response, 'html.parser')

In [ ]:
# Initialize the page number
page_number = 1
total_pages = int(soup.find_all('td', class_='left')[3].text.split(' ')[6].rstrip())

In [ ]:
page_number

1

In [ ]:
total_pages

10

In [3]:
data = []
for i in range(1, total_pages+1):
  url = f"https://stats.espncricinfo.com/ci/engine/stats/index.html?class=3;filter=advanced;orderby=wickets;page={i};size=200;spanmin1=1+Jan+2022;spanval1=span;template=results;type=bowling"
  response = requests.get(url)
  response = response.content
  soup = BeautifulSoup(response, 'html.parser')
  # Find the table element that contains the data we want to extract
  table = soup.select_one('#ciHomeContentlhs > div.pnl650M > table:nth-child(5)')

  # Check if the table exists
  if table is not None:
    # Loop through the rows of the table and extract the data for each player
    for row in table.tbody.find_all('tr'):
      columns = row.find_all('td')
      if len(columns) > 0:
        Player = columns[0].text.strip()
        Span = columns[1].text.strip()
        Matches = columns[2].text.strip()
        Innings = columns[3].text.strip()
        Overs = columns[4].text.strip()
        Maidens = columns[5].text.strip()
        Runs = columns[6].text.strip()
        Wickets = columns[7].text.strip()
        BBI = columns[8].text.strip()
        Average = columns[9].text.strip()
        Economy = columns[10].text.strip()
        SR = columns[11].text.strip()
        Four_Wickets = columns[12].text.strip()
        Five_Wickets = columns[13].text.strip()


        # Add the data for each player to the DataFrame
        data.append({
          'Player': Player,
          'Span': Span,
          'Matches': Matches,
          'Innings': Innings,
          'Overs' : Overs,
          'Maidens': Maidens,
          'Runs': Runs,
          'Wickets' : Wickets,
          'BBI': BBI,
          'Average': Average,
          'Economy': Economy,
          'SR': SR,
          'Four_Wickets' : Four_Wickets,
          'Five_Wickets': Five_Wickets
        })

    # Check if there is a next page by finding the 'Next' button on the page
    next_page_link = soup.select_one('.PaginationLink')
    if next_page_link is not None:
      # If there is a next page, increment the page number and update the URL to the next page
      i += 1
    else:
      # If there is no next page, break out of the loop
      break

# Create a DataFrame from the scraped data
df4 = pd.DataFrame(data)

In [4]:
# Print the DataFrame
df4.head(10)

,Overs,Maidens,Runs,Wickets,Economy,Inns,Opposition,Ground,Start_Date
0,Hasan Mahmud,8.1,1,32,5,3.91,1,,v Ireland
1,Shakib Al Hasan,10.0,2,36,5,3.60,1,,v India
2,Mustafizur Rahman,5.2,0,17,4,3.18,2,,v Zimbabwe
3,Shoriful Islam,9.0,1,21,4,2.33,1,,v Afghanistan
4,Shakib Al Hasan,10.0,0,35,4,3.50,2,,v England
5,Ebadot Hossain,6.5,0,42,4,6.14,2,,v Ireland
6,Mustafizur Rahman,10.0,1,44,4,4.40,2,,v Ireland
7,Ebadot Hossain,8.2,0,47,4,5.64,1,,v India
8,Taskin Ahmed,10.0,1,26,3,2.60,1,,v Ireland
9,Nasum Ahmed,8.0,0,43,3,5.37,2,,v Ireland


In [ ]:
# Extract player name and country using regular expression
df4[['Player_Name', 'Country']] = df4['Player'].str.extract(r'^(.*?) \((.*?)\)')

In [ ]:
df4

,Player,Span,Matches,Innings,Overs,Maidens,Runs,Wickets,BBI,Average,Economy,SR,Four_Wickets,Five_Wickets,Player_Name,Country
0,Arshdeep Singh (IND),2022-2023,33,33,111.5,1,949,50,4/37,18.98,8.48,13.4,1,0,Arshdeep Singh,IND
1,YM Nkanya (TAN),2022-2023,34,32,113.0,4,536,48,5/2,11.16,4.74,14.1,2,1,YM Nkanya,TAN
2,MR Adair (IRE),2022-2023,40,40,136.2,0,1122,47,4/13,23.87,8.22,17.4,1,0,MR Adair,IRE
3,JB Little (IRE),2022-2023,33,33,124.2,1,943,45,4/35,20.95,7.58,16.5,1,0,JB Little,IRE
4,D Maisuria (BOT),2022-2023,23,22,80.4,3,421,44,5/18,9.56,5.21,11.0,2,2,D Maisuria,BOT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1915,Zhuang Zelin (CHN),2023-2023,4,-,-,-,-,-,-,-,-,-,-,-,Zhuang Zelin,CHN
1916,Zong Yuechao (CHN),2023-2023,1,-,-,-,-,-,-,-,-,-,-,-,Zong Yuechao,CHN
1917,Zou Kui (CHN),2023-2023,1,-,-,-,-,-,-,-,-,-,-,-,Zou Kui,CHN
1918,Zubaidi Zulkifle (MAL),2022-2023,32,-,-,-,-,-,-,-,-,-,-,-,Zubaidi Zulkifle,MAL


In [ ]:
df4 = df4.drop('Span', axis=1)
df4

,Player,Matches,Innings,Overs,Maidens,Runs,Wickets,BBI,Average,Economy,SR,Four_Wickets,Five_Wickets,Player_Name,Country
0,Arshdeep Singh (IND),33,33,111.5,1,949,50,4/37,18.98,8.48,13.4,1,0,Arshdeep Singh,IND
1,YM Nkanya (TAN),34,32,113.0,4,536,48,5/2,11.16,4.74,14.1,2,1,YM Nkanya,TAN
2,MR Adair (IRE),40,40,136.2,0,1122,47,4/13,23.87,8.22,17.4,1,0,MR Adair,IRE
3,JB Little (IRE),33,33,124.2,1,943,45,4/35,20.95,7.58,16.5,1,0,JB Little,IRE
4,D Maisuria (BOT),23,22,80.4,3,421,44,5/18,9.56,5.21,11.0,2,2,D Maisuria,BOT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1915,Zhuang Zelin (CHN),4,-,-,-,-,-,-,-,-,-,-,-,Zhuang Zelin,CHN
1916,Zong Yuechao (CHN),1,-,-,-,-,-,-,-,-,-,-,-,Zong Yuechao,CHN
1917,Zou Kui (CHN),1,-,-,-,-,-,-,-,-,-,-,-,Zou Kui,CHN
1918,Zubaidi Zulkifle (MAL),32,-,-,-,-,-,-,-,-,-,-,-,Zubaidi Zulkifle,MAL


In [ ]:
def replace_values(value):
    if value == 'DNB':
      return 0
    elif value == 'TDNB':
      return 0
    elif value == 'absent':
      return 0
    elif value == 'sub':
      return 0
    elif value == '-':
      return 0
    else:
      return value

# apply the replace_values function to all columns in the DataFrame
df4 = df4.applymap(replace_values)
df4

,Player,Matches,Innings,Overs,Maidens,Runs,Wickets,BBI,Average,Economy,SR,Four_Wickets,Five_Wickets,Player_Name,Country
0,Arshdeep Singh (IND),33,33,111.5,1,949,50,4/37,18.98,8.48,13.4,1,0,Arshdeep Singh,IND
1,YM Nkanya (TAN),34,32,113.0,4,536,48,5/2,11.16,4.74,14.1,2,1,YM Nkanya,TAN
2,MR Adair (IRE),40,40,136.2,0,1122,47,4/13,23.87,8.22,17.4,1,0,MR Adair,IRE
3,JB Little (IRE),33,33,124.2,1,943,45,4/35,20.95,7.58,16.5,1,0,JB Little,IRE
4,D Maisuria (BOT),23,22,80.4,3,421,44,5/18,9.56,5.21,11.0,2,2,D Maisuria,BOT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1915,Zhuang Zelin (CHN),4,0,0,0,0,0,0,0,0,0,0,0,Zhuang Zelin,CHN
1916,Zong Yuechao (CHN),1,0,0,0,0,0,0,0,0,0,0,0,Zong Yuechao,CHN
1917,Zou Kui (CHN),1,0,0,0,0,0,0,0,0,0,0,0,Zou Kui,CHN
1918,Zubaidi Zulkifle (MAL),32,0,0,0,0,0,0,0,0,0,0,0,Zubaidi Zulkifle,MAL


In [ ]:
df4.to_excel('Overall_bowlers.xlsx')

# Positionwise Batting Stats

## Upper Order

In [ ]:
# Define the base URL and the initial page URL
base_url = 'https://stats.espncricinfo.com'
url = base_url + '/ci/engine/stats/index.html?batting_positionmax2=3;batting_positionmin2=1;batting_positionval2=batting_position;class=3;filter=advanced;home_or_away=1;home_or_away=2;home_or_away=3;innings_number=1;innings_number=2;orderby=runs;page=1;size=200;spanmin1=1+Jan+2022;spanval1=span;template=results;type=batting'

response = requests.get(url)
response = response.content
soup = BeautifulSoup(response, 'html.parser')

In [ ]:
# Initialize the page number
page_number = 1
total_pages = int(soup.find_all('td', class_='left')[6].text.split(' ')[6].rstrip())

In [ ]:
page_number

1

In [ ]:
total_pages

4

In [ ]:
data = []
for i in range(1, total_pages+1):
  url = f"https://stats.espncricinfo.com/ci/engine/stats/index.html?batting_positionmax2=3;batting_positionmin2=1;batting_positionval2=batting_position;class=3;filter=advanced;home_or_away=1;home_or_away=2;home_or_away=3;innings_number=1;innings_number=2;orderby=runs;page={i};size=200;spanmin1=1+Jan+2022;spanval1=span;template=results;type=batting"
  response = requests.get(url)
  response = response.content
  soup = BeautifulSoup(response, 'html.parser')
  # Find the table element that contains the data we want to extract
  table = soup.select_one('#ciHomeContentlhs > div.pnl650M > table:nth-child(5)')

  # Check if the table exists
  if table is not None:
    # Loop through the rows of the table and extract the data for each player
    for row in table.tbody.find_all('tr'):
      columns = row.find_all('td')
      if len(columns) > 0:
        Player = columns[0].text.strip()
        Span = columns[1].text.strip()
        Matches = columns[2].text.strip()
        Innings = columns[3].text.strip()
        Not_Out = columns[4].text.strip()
        Runs = columns[5].text.strip()
        Highest_Score = columns[6].text.strip()
        Average = columns[7].text.strip()
        Balls_Faced = columns[8].text.strip()
        SR = columns[9].text.strip()
        Hundreds = columns[10].text.strip()
        Fifties = columns[11].text.strip()
        Zeros = columns[12].text.strip()
        Fours = columns[13].text.strip()
        Sixes = columns[14].text.strip()


        # Add the data for each player to the DataFrame
        data.append({
          'Player': Player,
          'Span': Span,
          'Matches': Matches,
          'Innings': Innings,
          'Not_Out' : Not_Out,
          'Runs': Runs,
          'Highest_Score': Highest_Score,
          'Average' : Average,
          'Balls_Faced': Balls_Faced,
          'SR': SR,
          'Hundreds': Hundreds,
          'Fifties': Fifties,
          'Zeros' : Zeros,
          'Fours': Fours,
          'Sixes': Sixes
        })

    # Check if there is a next page by finding the 'Next' button on the page
    next_page_link = soup.select_one('.PaginationLink')
    if next_page_link is not None:
      # If there is a next page, increment the page number and update the URL to the next page
      i += 1
    else:
      # If there is no next page, break out of the loop
      break

# Create a DataFrame from the scraped data
df5 = pd.DataFrame(data)

In [ ]:
# Print the DataFrame
df5.head(10)

,Player,Span,Matches,Innings,Not_Out,Runs,Highest_Score,Average,Balls_Faced,SR,Hundreds,Fifties,Zeros,Fours,Sixes
0,Mohammad Rizwan (PAK),2022-2023,29,29,4,1158,98*,46.32,926,125.05,0,12,0,93,27
1,S Ssesazi (UGA),2022-2023,37,37,4,960,100*,29.09,754,127.32,1,7,7,100,33
2,Virandeep Singh (MAL),2022-2023,27,27,6,897,116*,42.71,639,140.37,1,6,3,88,39
3,Babar Azam (PAK),2022-2023,30,30,4,865,110*,33.26,685,126.27,2,5,2,99,13
4,Muhammad Waseem (UAE),2022-2023,22,22,0,858,112,39.00,583,147.16,1,8,1,59,57
5,A Balbirnie (IRE),2022-2023,36,36,1,848,75*,24.22,668,126.94,0,6,2,78,41
6,Litton Das (BAN),2022-2023,26,26,0,817,83,31.42,573,142.58,0,6,0,91,23
7,PR Stirling (IRE),2022-2023,37,37,3,802,77,23.58,600,133.66,0,4,4,92,34
8,Zubaidi Zulkifle (MAL),2022-2023,31,31,2,801,96,27.62,491,163.13,0,3,4,82,43
9,P Nissanka (SL),2022-2023,30,30,1,796,75,27.44,721,110.40,0,6,3,75,17


In [ ]:
df5 = df5.drop('Span', axis=1)
df5

,Player,Matches,Innings,Not_Out,Runs,Highest_Score,Average,Balls_Faced,SR,Hundreds,Fifties,Zeros,Fours,Sixes
0,Mohammad Rizwan (PAK),29,29,4,1158,98*,46.32,926,125.05,0,12,0,93,27
1,S Ssesazi (UGA),37,37,4,960,100*,29.09,754,127.32,1,7,7,100,33
2,Virandeep Singh (MAL),27,27,6,897,116*,42.71,639,140.37,1,6,3,88,39
3,Babar Azam (PAK),30,30,4,865,110*,33.26,685,126.27,2,5,2,99,13
4,Muhammad Waseem (UAE),22,22,0,858,112,39.00,583,147.16,1,8,1,59,57
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
751,Khalid Zahid (SWE),1,-,-,-,-,-,-,-,-,-,-,-,-
752,Ghulam Ahmadi (GER),1,-,-,-,-,-,-,-,-,-,-,-,-
753,RK Kiseto (TAN),1,-,-,-,-,-,-,-,-,-,-,-,-
754,LN Oluoch (KENYA),2,-,-,-,-,-,-,-,-,-,-,-,-


In [ ]:
def replace_values(value):
    if value == 'DNB':
      return 0
    elif value == 'TDNB':
      return 0
    elif value == 'absent':
      return 0
    elif value == 'sub':
      return 0
    elif value == '-':
      return 0
    else:
      return value

# apply the replace_values function to all columns in the DataFrame
df5 = df5.applymap(replace_values)
df5

,Player,Matches,Innings,Not_Out,Runs,Highest_Score,Average,Balls_Faced,SR,Hundreds,Fifties,Zeros,Fours,Sixes
0,Mohammad Rizwan (PAK),29,29,4,1158,98*,46.32,926,125.05,0,12,0,93,27
1,S Ssesazi (UGA),37,37,4,960,100*,29.09,754,127.32,1,7,7,100,33
2,Virandeep Singh (MAL),27,27,6,897,116*,42.71,639,140.37,1,6,3,88,39
3,Babar Azam (PAK),30,30,4,865,110*,33.26,685,126.27,2,5,2,99,13
4,Muhammad Waseem (UAE),22,22,0,858,112,39.00,583,147.16,1,8,1,59,57
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
751,Khalid Zahid (SWE),1,0,0,0,0,0,0,0,0,0,0,0,0
752,Ghulam Ahmadi (GER),1,0,0,0,0,0,0,0,0,0,0,0,0
753,RK Kiseto (TAN),1,0,0,0,0,0,0,0,0,0,0,0,0
754,LN Oluoch (KENYA),2,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
# Extract player name and country using regular expression
df5[['Player_Name', 'Country']] = df5['Player'].str.extract(r'^(.*?) \((.*?)\)')

In [ ]:
df5

,Player,Matches,Innings,Not_Out,Runs,Highest_Score,Average,Balls_Faced,SR,Hundreds,Fifties,Zeros,Fours,Sixes,Player_Name,Country
0,Mohammad Rizwan (PAK),29,29,4,1158,98*,46.32,926,125.05,0,12,0,93,27,Mohammad Rizwan,PAK
1,S Ssesazi (UGA),37,37,4,960,100*,29.09,754,127.32,1,7,7,100,33,S Ssesazi,UGA
2,Virandeep Singh (MAL),27,27,6,897,116*,42.71,639,140.37,1,6,3,88,39,Virandeep Singh,MAL
3,Babar Azam (PAK),30,30,4,865,110*,33.26,685,126.27,2,5,2,99,13,Babar Azam,PAK
4,Muhammad Waseem (UAE),22,22,0,858,112,39.00,583,147.16,1,8,1,59,57,Muhammad Waseem,UAE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
751,Khalid Zahid (SWE),1,0,0,0,0,0,0,0,0,0,0,0,0,Khalid Zahid,SWE
752,Ghulam Ahmadi (GER),1,0,0,0,0,0,0,0,0,0,0,0,0,Ghulam Ahmadi,GER
753,RK Kiseto (TAN),1,0,0,0,0,0,0,0,0,0,0,0,0,RK Kiseto,TAN
754,LN Oluoch (KENYA),2,0,0,0,0,0,0,0,0,0,0,0,0,LN Oluoch,KENYA


In [ ]:
df5.to_excel('Upper_Order.xlsx')

## Middle-Order

In [ ]:
# Define the base URL and the initial page URL
base_url = 'https://stats.espncricinfo.com'
url = base_url + '/ci/engine/stats/index.html?batting_positionmax2=7;batting_positionmin2=4;batting_positionval2=batting_position;class=3;filter=advanced;home_or_away=1;home_or_away=2;home_or_away=3;innings_number=1;innings_number=2;orderby=runs;page=1;size=200;spanmin1=1+Jan+2022;spanval1=span;template=results;type=batting'

response = requests.get(url)
response = response.content
soup = BeautifulSoup(response, 'html.parser')

In [ ]:
# Initialize the page number
page_number = 1
total_pages = int(soup.find_all('td', class_='left')[6].text.split(' ')[6].rstrip())

In [ ]:
page_number

1

In [ ]:
total_pages

6

In [ ]:
data = []
for i in range(1, total_pages+1):
  url = f"https://stats.espncricinfo.com/ci/engine/stats/index.html?batting_positionmax2=7;batting_positionmin2=4;batting_positionval2=batting_position;class=3;filter=advanced;home_or_away=1;home_or_away=2;home_or_away=3;innings_number=1;innings_number=2;orderby=runs;page={i};size=200;spanmin1=1+Jan+2022;spanval1=span;template=results;type=batting"
  response = requests.get(url)
  response = response.content
  soup = BeautifulSoup(response, 'html.parser')
  # Find the table element that contains the data we want to extract
  table = soup.select_one('#ciHomeContentlhs > div.pnl650M > table:nth-child(5)')

  # Check if the table exists
  if table is not None:
    # Loop through the rows of the table and extract the data for each player
    for row in table.tbody.find_all('tr'):
      columns = row.find_all('td')
      if len(columns) > 0:
        Player = columns[0].text.strip()
        Span = columns[1].text.strip()
        Matches = columns[2].text.strip()
        Innings = columns[3].text.strip()
        Not_Out = columns[4].text.strip()
        Runs = columns[5].text.strip()
        Highest_Score = columns[6].text.strip()
        Average = columns[7].text.strip()
        Balls_Faced = columns[8].text.strip()
        SR = columns[9].text.strip()
        Hundreds = columns[10].text.strip()
        Fifties = columns[11].text.strip()
        Zeros = columns[12].text.strip()
        Fours = columns[13].text.strip()
        Sixes = columns[14].text.strip()


        # Add the data for each player to the DataFrame
        data.append({
          'Player': Player,
          'Span': Span,
          'Matches': Matches,
          'Innings': Innings,
          'Not_Out' : Not_Out,
          'Runs': Runs,
          'Highest_Score': Highest_Score,
          'Average' : Average,
          'Balls_Faced': Balls_Faced,
          'SR': SR,
          'Hundreds': Hundreds,
          'Fifties': Fifties,
          'Zeros' : Zeros,
          'Fours': Fours,
          'Sixes': Sixes
        })

    # Check if there is a next page by finding the 'Next' button on the page
    next_page_link = soup.select_one('.PaginationLink')
    if next_page_link is not None:
      # If there is a next page, increment the page number and update the URL to the next page
      i += 1
    else:
      # If there is no next page, break out of the loop
      break

# Create a DataFrame from the scraped data
df5 = pd.DataFrame(data)

In [ ]:
# Print the DataFrame
df5.head(10)

,Player,Span,Matches,Innings,Not_Out,Runs,Highest_Score,Average,Balls_Faced,SR,Hundreds,Fifties,Zeros,Fours,Sixes
0,SA Yadav (IND),2022-2023,29,28,7,1159,117,55.19,642,180.52,2,9,1,100,67
1,HH Pandya (IND),2022-2023,38,35,9,795,71*,30.57,586,135.66,0,3,1,62,37
2,Sikandar Raza (ZIM),2022-2022,24,23,2,735,87,35.00,487,150.92,0,5,4,52,38
3,GD Phillips (NZ),2022-2023,23,21,3,717,104,39.83,481,149.06,1,6,0,51,32
4,R Powell (WI),2022-2023,29,28,6,705,107,32.04,458,153.93,1,2,0,31,54
5,VP Thamotharam (MLT),2022-2023,29,29,4,649,62,25.96,432,150.23,0,5,1,43,43
6,DJ Mitchell (NZ),2022-2023,27,26,5,648,66,30.85,464,139.65,0,4,0,37,26
7,MD Shanaka (SL),2022-2023,31,30,7,644,74*,28.00,451,142.79,0,3,3,48,35
8,Sohail Ahmed (BHR),2022-2023,18,18,6,625,80*,52.08,484,129.13,0,6,1,43,27
9,HT Tector (IRE),2022-2023,36,33,5,621,64*,22.17,531,116.94,0,2,1,46,16


In [ ]:
df5 = df5.drop('Span', axis=1)
df5

,Player,Matches,Innings,Not_Out,Runs,Highest_Score,Average,Balls_Faced,SR,Hundreds,Fifties,Zeros,Fours,Sixes
0,SA Yadav (IND),29,28,7,1159,117,55.19,642,180.52,2,9,1,100,67
1,HH Pandya (IND),38,35,9,795,71*,30.57,586,135.66,0,3,1,62,37
2,Sikandar Raza (ZIM),24,23,2,735,87,35.00,487,150.92,0,5,4,52,38
3,GD Phillips (NZ),23,21,3,717,104,39.83,481,149.06,1,6,0,51,32
4,R Powell (WI),29,28,6,705,107,32.04,458,153.93,1,2,0,31,54
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1156,Vraj Patel (KENYA),5,-,-,-,-,-,-,-,-,-,-,-,-
1157,DKD Warnakulasuriya (ITA),1,-,-,-,-,-,-,-,-,-,-,-,-
1158,Waseeq Ahmed (BHR),1,-,-,-,-,-,-,-,-,-,-,-,-
1159,SMK Waththage (CZK-R),1,-,-,-,-,-,-,-,-,-,-,-,-


In [ ]:
def replace_values(value):
    if value == 'DNB':
      return 0
    elif value == 'TDNB':
      return 0
    elif value == 'absent':
      return 0
    elif value == 'sub':
      return 0
    elif value == '-':
      return 0
    else:
      return value

# apply the replace_values function to all columns in the DataFrame
df5 = df5.applymap(replace_values)
df5

,Player,Matches,Innings,Not_Out,Runs,Highest_Score,Average,Balls_Faced,SR,Hundreds,Fifties,Zeros,Fours,Sixes
0,SA Yadav (IND),29,28,7,1159,117,55.19,642,180.52,2,9,1,100,67
1,HH Pandya (IND),38,35,9,795,71*,30.57,586,135.66,0,3,1,62,37
2,Sikandar Raza (ZIM),24,23,2,735,87,35.00,487,150.92,0,5,4,52,38
3,GD Phillips (NZ),23,21,3,717,104,39.83,481,149.06,1,6,0,51,32
4,R Powell (WI),29,28,6,705,107,32.04,458,153.93,1,2,0,31,54
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1156,Vraj Patel (KENYA),5,0,0,0,0,0,0,0,0,0,0,0,0
1157,DKD Warnakulasuriya (ITA),1,0,0,0,0,0,0,0,0,0,0,0,0
1158,Waseeq Ahmed (BHR),1,0,0,0,0,0,0,0,0,0,0,0,0
1159,SMK Waththage (CZK-R),1,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
# Extract player name and country using regular expression
df5[['Player_Name', 'Country']] = df5['Player'].str.extract(r'^(.*?) \((.*?)\)')

In [ ]:
df5

,Player,Matches,Innings,Not_Out,Runs,Highest_Score,Average,Balls_Faced,SR,Hundreds,Fifties,Zeros,Fours,Sixes,Player_Name,Country
0,SA Yadav (IND),29,28,7,1159,117,55.19,642,180.52,2,9,1,100,67,SA Yadav,IND
1,HH Pandya (IND),38,35,9,795,71*,30.57,586,135.66,0,3,1,62,37,HH Pandya,IND
2,Sikandar Raza (ZIM),24,23,2,735,87,35.00,487,150.92,0,5,4,52,38,Sikandar Raza,ZIM
3,GD Phillips (NZ),23,21,3,717,104,39.83,481,149.06,1,6,0,51,32,GD Phillips,NZ
4,R Powell (WI),29,28,6,705,107,32.04,458,153.93,1,2,0,31,54,R Powell,WI
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1156,Vraj Patel (KENYA),5,0,0,0,0,0,0,0,0,0,0,0,0,Vraj Patel,KENYA
1157,DKD Warnakulasuriya (ITA),1,0,0,0,0,0,0,0,0,0,0,0,0,DKD Warnakulasuriya,ITA
1158,Waseeq Ahmed (BHR),1,0,0,0,0,0,0,0,0,0,0,0,0,Waseeq Ahmed,BHR
1159,SMK Waththage (CZK-R),1,0,0,0,0,0,0,0,0,0,0,0,0,SMK Waththage,CZK-R


In [ ]:
df5.to_excel('Middle_Order.xlsx')

## Lower Order

In [ ]:
# Define the base URL and the initial page URL
base_url = 'https://stats.espncricinfo.com'
url = base_url + '/ci/engine/stats/index.html?batting_positionmax2=11;batting_positionmin2=8;batting_positionval2=batting_position;class=3;filter=advanced;home_or_away=1;home_or_away=2;home_or_away=3;innings_number=1;innings_number=2;orderby=runs;page=1;size=200;spanmin1=1+Jan+2022;spanval1=span;template=results;type=batting'

response = requests.get(url)
response = response.content
soup = BeautifulSoup(response, 'html.parser')

In [ ]:
# Initialize the page number
page_number = 1
total_pages = int(soup.find_all('td', class_='left')[6].text.split(' ')[6].rstrip())

In [ ]:
page_number

1

In [ ]:
total_pages

7

In [ ]:
data = []
for i in range(1, total_pages+1):
  url = f"https://stats.espncricinfo.com/ci/engine/stats/index.html?batting_positionmax2=11;batting_positionmin2=8;batting_positionval2=batting_position;class=3;filter=advanced;home_or_away=1;home_or_away=2;home_or_away=3;innings_number=1;innings_number=2;orderby=runs;page={i};size=200;spanmin1=1+Jan+2022;spanval1=span;template=results;type=batting"
  response = requests.get(url)
  response = response.content
  soup = BeautifulSoup(response, 'html.parser')
  # Find the table element that contains the data we want to extract
  table = soup.select_one('#ciHomeContentlhs > div.pnl650M > table:nth-child(5)')

  # Check if the table exists
  if table is not None:
    # Loop through the rows of the table and extract the data for each player
    for row in table.tbody.find_all('tr'):
      columns = row.find_all('td')
      if len(columns) > 0:
        Player = columns[0].text.strip()
        Span = columns[1].text.strip()
        Matches = columns[2].text.strip()
        Innings = columns[3].text.strip()
        Not_Out = columns[4].text.strip()
        Runs = columns[5].text.strip()
        Highest_Score = columns[6].text.strip()
        Average = columns[7].text.strip()
        Balls_Faced = columns[8].text.strip()
        SR = columns[9].text.strip()
        Hundreds = columns[10].text.strip()
        Fifties = columns[11].text.strip()
        Zeros = columns[12].text.strip()
        Fours = columns[13].text.strip()
        Sixes = columns[14].text.strip()


        # Add the data for each player to the DataFrame
        data.append({
          'Player': Player,
          'Span': Span,
          'Matches': Matches,
          'Innings': Innings,
          'Not_Out' : Not_Out,
          'Runs': Runs,
          'Highest_Score': Highest_Score,
          'Average' : Average,
          'Balls_Faced': Balls_Faced,
          'SR': SR,
          'Hundreds': Hundreds,
          'Fifties': Fifties,
          'Zeros' : Zeros,
          'Fours': Fours,
          'Sixes': Sixes
        })

    # Check if there is a next page by finding the 'Next' button on the page
    next_page_link = soup.select_one('.PaginationLink')
    if next_page_link is not None:
      # If there is a next page, increment the page number and update the URL to the next page
      i += 1
    else:
      # If there is no next page, break out of the loop
      break

# Create a DataFrame from the scraped data
df5 = pd.DataFrame(data)

In [ ]:
# Print the DataFrame
df5.head(10)

,Player,Span,Matches,Innings,Not_Out,Runs,Highest_Score,Average,Balls_Faced,SR,Hundreds,Fifties,Zeros,Fours,Sixes
0,MR Adair (IRE),2022-2023,33,20,3,263,72,15.47,205,128.29,0,1,1,17,14
1,BJ McCarthy (IRE),2022-2023,27,18,6,195,51*,16.25,143,136.36,0,1,1,14,12
2,LM Jongwe (ZIM),2022-2023,20,15,5,176,35,17.60,129,136.43,0,0,2,15,7
3,Muslim Yar (GER),2022-2023,20,11,5,160,39,26.66,112,142.85,0,0,1,11,8
4,C Karunaratne (SL),2022-2023,20,16,7,144,31,16.00,138,104.34,0,0,4,11,4
5,Rashid Khan (AFG),2022-2023,19,13,5,136,48*,17.00,111,122.52,0,0,1,10,8
6,F Nsubuga (UGA),2022-2023,27,14,3,125,31*,11.36,114,109.64,0,0,3,6,7
7,R Shepherd (WI),2022-2023,10,4,2,122,44*,61.00,77,158.44,0,0,0,4,10
8,K Irakoze (RWN),2022-2023,25,17,3,118,23,8.42,138,85.50,0,0,5,6,4
9,Naseem Khushi (OMA),2022-2022,7,5,1,116,47*,29.00,79,146.83,0,0,0,5,9


In [ ]:
df5 = df5.drop('Span', axis=1)
df5

,Player,Matches,Innings,Not_Out,Runs,Highest_Score,Average,Balls_Faced,SR,Hundreds,Fifties,Zeros,Fours,Sixes
0,MR Adair (IRE),33,20,3,263,72,15.47,205,128.29,0,1,1,17,14
1,BJ McCarthy (IRE),27,18,6,195,51*,16.25,143,136.36,0,1,1,14,12
2,LM Jongwe (ZIM),20,15,5,176,35,17.60,129,136.43,0,0,2,15,7
3,Muslim Yar (GER),20,11,5,160,39,26.66,112,142.85,0,0,1,11,8
4,C Karunaratne (SL),20,16,7,144,31,16.00,138,104.34,0,0,4,11,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1229,Zeeshan Abbas (BHR),3,-,-,-,-,-,-,-,-,-,-,-,-
1230,Zeeshan Ali (HKG),2,-,-,-,-,-,-,-,-,-,-,-,-
1231,Zeeshan Maqsood (OMA),2,-,-,-,-,-,-,-,-,-,-,-,-
1232,Zou Kui (CHN),1,-,-,-,-,-,-,-,-,-,-,-,-


In [ ]:
def replace_values(value):
    if value == 'DNB':
      return 0
    elif value == 'TDNB':
      return 0
    elif value == 'absent':
      return 0
    elif value == 'sub':
      return 0
    elif value == '-':
      return 0
    else:
      return value

# apply the replace_values function to all columns in the DataFrame
df5 = df5.applymap(replace_values)
df5

,Player,Matches,Innings,Not_Out,Runs,Highest_Score,Average,Balls_Faced,SR,Hundreds,Fifties,Zeros,Fours,Sixes
0,MR Adair (IRE),33,20,3,263,72,15.47,205,128.29,0,1,1,17,14
1,BJ McCarthy (IRE),27,18,6,195,51*,16.25,143,136.36,0,1,1,14,12
2,LM Jongwe (ZIM),20,15,5,176,35,17.60,129,136.43,0,0,2,15,7
3,Muslim Yar (GER),20,11,5,160,39,26.66,112,142.85,0,0,1,11,8
4,C Karunaratne (SL),20,16,7,144,31,16.00,138,104.34,0,0,4,11,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1229,Zeeshan Abbas (BHR),3,0,0,0,0,0,0,0,0,0,0,0,0
1230,Zeeshan Ali (HKG),2,0,0,0,0,0,0,0,0,0,0,0,0
1231,Zeeshan Maqsood (OMA),2,0,0,0,0,0,0,0,0,0,0,0,0
1232,Zou Kui (CHN),1,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
# Extract player name and country using regular expression
df5[['Player_Name', 'Country']] = df5['Player'].str.extract(r'^(.*?) \((.*?)\)')

In [ ]:
df5

,Player,Matches,Innings,Not_Out,Runs,Highest_Score,Average,Balls_Faced,SR,Hundreds,Fifties,Zeros,Fours,Sixes,Player_Name,Country
0,MR Adair (IRE),33,20,3,263,72,15.47,205,128.29,0,1,1,17,14,MR Adair,IRE
1,BJ McCarthy (IRE),27,18,6,195,51*,16.25,143,136.36,0,1,1,14,12,BJ McCarthy,IRE
2,LM Jongwe (ZIM),20,15,5,176,35,17.60,129,136.43,0,0,2,15,7,LM Jongwe,ZIM
3,Muslim Yar (GER),20,11,5,160,39,26.66,112,142.85,0,0,1,11,8,Muslim Yar,GER
4,C Karunaratne (SL),20,16,7,144,31,16.00,138,104.34,0,0,4,11,4,C Karunaratne,SL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1229,Zeeshan Abbas (BHR),3,0,0,0,0,0,0,0,0,0,0,0,0,Zeeshan Abbas,BHR
1230,Zeeshan Ali (HKG),2,0,0,0,0,0,0,0,0,0,0,0,0,Zeeshan Ali,HKG
1231,Zeeshan Maqsood (OMA),2,0,0,0,0,0,0,0,0,0,0,0,0,Zeeshan Maqsood,OMA
1232,Zou Kui (CHN),1,0,0,0,0,0,0,0,0,0,0,0,0,Zou Kui,CHN


In [ ]:
df5.to_excel('Lower_Order.xlsx')

# Positionwise Bowling Stats

## Opening

In [ ]:
# Define the initial page URL
url = 'https://stats.espncricinfo.com/ci/engine/stats/index.html?bowling_positionmax1=2;bowling_positionmax2=2;bowling_positionmin1=1;bowling_positionmin2=1;bowling_positionval1=bowling_position;bowling_positionval2=bowling_position;class=3;filter=advanced;home_or_away=1;home_or_away=2;home_or_away=3;innings_number=1;innings_number=2;orderby=wickets;page=1;size=200;spanmin1=1+Jan+2022;spanval1=span;template=results;type=bowling'

response = requests.get(url)
response = response.content
soup = BeautifulSoup(response, 'html.parser')

In [ ]:
# Initialize the page number
page_number = 1
total_pages = int(soup.find_all('td', class_='left')[6].text.split(' ')[6].rstrip())

In [ ]:
page_number

1

In [ ]:
total_pages

4

In [ ]:
data = []
for i in range(1, total_pages+1):
  url = f"https://stats.espncricinfo.com/ci/engine/stats/index.html?bowling_positionmax1=2;bowling_positionmax2=2;bowling_positionmin1=1;bowling_positionmin2=1;bowling_positionval1=bowling_position;bowling_positionval2=bowling_position;class=3;filter=advanced;home_or_away=1;home_or_away=2;home_or_away=3;innings_number=1;innings_number=2;orderby=wickets;page={i};size=200;spanmin1=1+Jan+2022;spanval1=span;template=results;type=bowling"
  response = requests.get(url)
  response = response.content
  soup = BeautifulSoup(response, 'html.parser')
  # Find the table element that contains the data we want to extract
  table = soup.select_one('#ciHomeContentlhs > div.pnl650M > table:nth-child(5)')

  # Check if the table exists
  if table is not None:
    # Loop through the rows of the table and extract the data for each player
    for row in table.tbody.find_all('tr'):
      columns = row.find_all('td')
      if len(columns) > 0:
        Player = columns[0].text.strip()
        Span = columns[1].text.strip()
        Matches = columns[2].text.strip()
        Innings = columns[3].text.strip()
        Overs = columns[4].text.strip()
        Maidens = columns[5].text.strip()
        Runs = columns[6].text.strip()
        Wickets = columns[7].text.strip()
        BBI = columns[8].text.strip()
        Average = columns[9].text.strip()
        Economy = columns[10].text.strip()
        SR = columns[11].text.strip()
        Four_Wickets = columns[12].text.strip()
        Five_Wickets = columns[13].text.strip()


        # Add the data for each player to the DataFrame
        data.append({
          'Player': Player,
          'Span': Span,
          'Matches': Matches,
          'Innings': Innings,
          'Overs' : Overs,
          'Maidens': Maidens,
          'Runs': Runs,
          'Wickets' : Wickets,
          'BBI': BBI,
          'Average': Average,
          'Economy': Economy,
          'SR': SR,
          'Four_Wickets' : Four_Wickets,
          'Five_Wickets': Five_Wickets
        })

    # Check if there is a next page by finding the 'Next' button on the page
    next_page_link = soup.select_one('.PaginationLink')
    if next_page_link is not None:
      # If there is a next page, increment the page number and update the URL to the next page
      i += 1
    else:
      # If there is no next page, break out of the loop
      break

# Create a DataFrame from the scraped data
df4 = pd.DataFrame(data)

In [ ]:
# Print the DataFrame
df4.head(10)

,Player,Span,Matches,Innings,Overs,Maidens,Runs,Wickets,BBI,Average,Economy,SR,Four_Wickets,Five_Wickets
0,MR Adair (IRE),2022-2023,40,40,136.2,0,1122,47,4/13,23.87,8.22,17.4,1,0
1,Arshdeep Singh (IND),2022-2023,29,29,100.4,1,890,43,4/37,20.69,8.84,14.0,1,0
2,B Kumar (IND),2022-2022,31,31,103.4,7,724,37,5/4,19.56,6.98,16.8,2,1
3,JB Little (IRE),2022-2023,28,28,104.2,1,769,35,3/22,21.97,7.37,17.8,0,0
4,TG Southee (NZ),2022-2023,17,17,64.0,3,494,28,5/25,17.64,7.71,13.7,0,1
5,Rizwan Butt (BHR),2022-2023,17,17,63.0,1,485,27,5/16,17.96,7.69,14.0,0,1
6,Fazalhaq Farooqi (AFG),2022-2023,23,23,86.4,3,563,26,3/11,21.65,6.49,20.0,0,0
7,JR Hazlewood (AUS),2022-2022,17,17,63.0,2,474,26,4/12,18.23,7.52,14.5,2,0
8,Z Bimenyimana (RWN),2022-2023,27,27,75.5,3,531,25,3/29,21.24,7.00,18.2,0,0
9,Taskin Ahmed (BAN),2022-2023,19,19,73.0,1,561,25,4/25,22.44,7.68,17.5,1,0


In [ ]:
# Extract player name and country using regular expression
df4[['Player_Name', 'Country']] = df4['Player'].str.extract(r'^(.*?) \((.*?)\)')

In [ ]:
df4

,Player,Span,Matches,Innings,Overs,Maidens,Runs,Wickets,BBI,Average,Economy,SR,Four_Wickets,Five_Wickets,Player_Name,Country
0,MR Adair (IRE),2022-2023,40,40,136.2,0,1122,47,4/13,23.87,8.22,17.4,1,0,MR Adair,IRE
1,Arshdeep Singh (IND),2022-2023,29,29,100.4,1,890,43,4/37,20.69,8.84,14.0,1,0,Arshdeep Singh,IND
2,B Kumar (IND),2022-2022,31,31,103.4,7,724,37,5/4,19.56,6.98,16.8,2,1,B Kumar,IND
3,JB Little (IRE),2022-2023,28,28,104.2,1,769,35,3/22,21.97,7.37,17.8,0,0,JB Little,IRE
4,TG Southee (NZ),2022-2023,17,17,64.0,3,494,28,5/25,17.64,7.71,13.7,0,1,TG Southee,NZ
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
610,AG Weligamage (HUN),2022-2022,2,2,8.0,0,76,0,-,-,9.50,-,0,0,AG Weligamage,HUN
611,S Wickramasekara (CZK-R),2022-2022,1,1,1.0,0,12,0,-,-,12.00,-,0,0,S Wickramasekara,CZK-R
612,S Williams (SLE),2022-2022,1,1,1.0,0,10,0,-,-,10.00,-,0,0,S Williams,SLE
613,S Yeshey (BHU),2023-2023,1,1,3.0,0,12,0,-,-,4.00,-,0,0,S Yeshey,BHU


In [ ]:
df4 = df4.drop('Span', axis=1)
df4

,Player,Matches,Innings,Overs,Maidens,Runs,Wickets,BBI,Average,Economy,SR,Four_Wickets,Five_Wickets,Player_Name,Country
0,MR Adair (IRE),40,40,136.2,0,1122,47,4/13,23.87,8.22,17.4,1,0,MR Adair,IRE
1,Arshdeep Singh (IND),29,29,100.4,1,890,43,4/37,20.69,8.84,14.0,1,0,Arshdeep Singh,IND
2,B Kumar (IND),31,31,103.4,7,724,37,5/4,19.56,6.98,16.8,2,1,B Kumar,IND
3,JB Little (IRE),28,28,104.2,1,769,35,3/22,21.97,7.37,17.8,0,0,JB Little,IRE
4,TG Southee (NZ),17,17,64.0,3,494,28,5/25,17.64,7.71,13.7,0,1,TG Southee,NZ
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
610,AG Weligamage (HUN),2,2,8.0,0,76,0,-,-,9.50,-,0,0,AG Weligamage,HUN
611,S Wickramasekara (CZK-R),1,1,1.0,0,12,0,-,-,12.00,-,0,0,S Wickramasekara,CZK-R
612,S Williams (SLE),1,1,1.0,0,10,0,-,-,10.00,-,0,0,S Williams,SLE
613,S Yeshey (BHU),1,1,3.0,0,12,0,-,-,4.00,-,0,0,S Yeshey,BHU


In [ ]:
def replace_values(value):
    if value == 'DNB':
      return 0
    elif value == 'TDNB':
      return 0
    elif value == 'absent':
      return 0
    elif value == 'sub':
      return 0
    elif value == '-':
      return 0
    else:
      return value

# apply the replace_values function to all columns in the DataFrame
df4 = df4.applymap(replace_values)
df4

,Player,Matches,Innings,Overs,Maidens,Runs,Wickets,BBI,Average,Economy,SR,Four_Wickets,Five_Wickets,Player_Name,Country
0,MR Adair (IRE),40,40,136.2,0,1122,47,4/13,23.87,8.22,17.4,1,0,MR Adair,IRE
1,Arshdeep Singh (IND),29,29,100.4,1,890,43,4/37,20.69,8.84,14.0,1,0,Arshdeep Singh,IND
2,B Kumar (IND),31,31,103.4,7,724,37,5/4,19.56,6.98,16.8,2,1,B Kumar,IND
3,JB Little (IRE),28,28,104.2,1,769,35,3/22,21.97,7.37,17.8,0,0,JB Little,IRE
4,TG Southee (NZ),17,17,64.0,3,494,28,5/25,17.64,7.71,13.7,0,1,TG Southee,NZ
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
610,AG Weligamage (HUN),2,2,8.0,0,76,0,0,0,9.50,0,0,0,AG Weligamage,HUN
611,S Wickramasekara (CZK-R),1,1,1.0,0,12,0,0,0,12.00,0,0,0,S Wickramasekara,CZK-R
612,S Williams (SLE),1,1,1.0,0,10,0,0,0,10.00,0,0,0,S Williams,SLE
613,S Yeshey (BHU),1,1,3.0,0,12,0,0,0,4.00,0,0,0,S Yeshey,BHU


In [ ]:
df4.to_excel('Opening_Bowlers.xlsx')

## First Change

In [ ]:
# Define the initial page URL
url = 'https://stats.espncricinfo.com/ci/engine/stats/index.html?bowling_positionmax2=3;bowling_positionmin2=3;bowling_positionval2=bowling_position;class=3;filter=advanced;home_or_away=1;home_or_away=2;home_or_away=3;innings_number=1;innings_number=2;orderby=wickets;page=1;size=200;spanmin1=1+Jan+2022;spanval1=span;template=results;type=bowling'

response = requests.get(url)
response = response.content
soup = BeautifulSoup(response, 'html.parser')

In [ ]:
# Initialize the page number
page_number = 1
total_pages = int(soup.find_all('td', class_='left')[6].text.split(' ')[6].rstrip())

In [ ]:
page_number

1

In [ ]:
total_pages

3

In [ ]:
data = []
for i in range(1, total_pages+1):
  url = f"https://stats.espncricinfo.com/ci/engine/stats/index.html?bowling_positionmax2=3;bowling_positionmin2=3;bowling_positionval2=bowling_position;class=3;filter=advanced;home_or_away=1;home_or_away=2;home_or_away=3;innings_number=1;innings_number=2;orderby=wickets;page={i};size=200;spanmin1=1+Jan+2022;spanval1=span;template=results;type=bowling"
  response = requests.get(url)
  response = response.content
  soup = BeautifulSoup(response, 'html.parser')
  # Find the table element that contains the data we want to extract
  table = soup.select_one('#ciHomeContentlhs > div.pnl650M > table:nth-child(5)')

  # Check if the table exists
  if table is not None:
    # Loop through the rows of the table and extract the data for each player
    for row in table.tbody.find_all('tr'):
      columns = row.find_all('td')
      if len(columns) > 0:
        Player = columns[0].text.strip()
        Span = columns[1].text.strip()
        Matches = columns[2].text.strip()
        Innings = columns[3].text.strip()
        Overs = columns[4].text.strip()
        Maidens = columns[5].text.strip()
        Runs = columns[6].text.strip()
        Wickets = columns[7].text.strip()
        BBI = columns[8].text.strip()
        Average = columns[9].text.strip()
        Economy = columns[10].text.strip()
        SR = columns[11].text.strip()
        Four_Wickets = columns[12].text.strip()
        Five_Wickets = columns[13].text.strip()


        # Add the data for each player to the DataFrame
        data.append({
          'Player': Player,
          'Span': Span,
          'Matches': Matches,
          'Innings': Innings,
          'Overs' : Overs,
          'Maidens': Maidens,
          'Runs': Runs,
          'Wickets' : Wickets,
          'BBI': BBI,
          'Average': Average,
          'Economy': Economy,
          'SR': SR,
          'Four_Wickets' : Four_Wickets,
          'Five_Wickets': Five_Wickets
        })

    # Check if there is a next page by finding the 'Next' button on the page
    next_page_link = soup.select_one('.PaginationLink')
    if next_page_link is not None:
      # If there is a next page, increment the page number and update the URL to the next page
      i += 1
    else:
      # If there is no next page, break out of the loop
      break

# Create a DataFrame from the scraped data
df4 = pd.DataFrame(data)

In [ ]:
# Print the DataFrame
df4.head(10)

,Player,Span,Matches,Innings,Overs,Maidens,Runs,Wickets,BBI,Average,Economy,SR,Four_Wickets,Five_Wickets
0,BJ McCarthy (IRE),2022-2023,21,21,75.4,1,605,22,3/7,27.50,7.99,20.6,0,0
1,AS Joseph (WI),2022-2023,9,9,34.0,0,253,18,5/40,14.05,7.44,11.3,1,1
2,CA Young (IRE),2022-2022,8,8,30.0,0,210,18,4/28,11.66,7.00,10.0,1,0
3,M Akayezu (RWN),2022-2023,13,13,40.0,2,297,17,3/27,17.47,7.42,14.1,0,0
4,L Ngidi (SA),2022-2023,10,10,33.0,0,324,17,5/39,19.05,9.81,11.6,1,1
5,MA Wood (ENG),2022-2022,6,6,23.0,0,161,17,3/20,9.47,7.00,8.1,0,0
6,S Veerapathiran (BHR),2022-2023,11,11,38.5,0,320,16,3/43,20.00,8.24,14.5,0,0
7,Haris Rauf (PAK),2022-2022,11,11,42.1,0,377,15,3/29,25.13,8.94,16.8,0,0
8,AM Kimote (TAN),2022-2023,12,12,33.2,0,239,15,4/21,15.93,7.17,13.3,1,0
9,Sami Sohail (MWI),2022-2023,9,9,32.0,2,187,15,4/6,12.46,5.84,12.8,1,0


In [ ]:
# Extract player name and country using regular expression
df4[['Player_Name', 'Country']] = df4['Player'].str.extract(r'^(.*?) \((.*?)\)')

In [ ]:
df4

,Player,Span,Matches,Innings,Overs,Maidens,Runs,Wickets,BBI,Average,Economy,SR,Four_Wickets,Five_Wickets,Player_Name,Country
0,BJ McCarthy (IRE),2022-2023,21,21,75.4,1,605,22,3/7,27.50,7.99,20.6,0,0,BJ McCarthy,IRE
1,AS Joseph (WI),2022-2023,9,9,34.0,0,253,18,5/40,14.05,7.44,11.3,1,1,AS Joseph,WI
2,CA Young (IRE),2022-2022,8,8,30.0,0,210,18,4/28,11.66,7.00,10.0,1,0,CA Young,IRE
3,M Akayezu (RWN),2022-2023,13,13,40.0,2,297,17,3/27,17.47,7.42,14.1,0,0,M Akayezu,RWN
4,L Ngidi (SA),2022-2023,10,10,33.0,0,324,17,5/39,19.05,9.81,11.6,1,1,L Ngidi,SA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
581,A Yon (STHEL),2022-2022,1,1,1.0,0,11,0,-,-,11.00,-,0,0,A Yon,STHEL
582,Zabiullah Zahid (SWE),2023-2023,1,1,4.0,0,34,0,-,-,8.50,-,0,0,Zabiullah Zahid,SWE
583,Zain Ul Abidin (Saudi),2022-2022,1,1,4.0,0,22,0,-,-,5.50,-,0,0,Zain Ul Abidin,Saudi
584,V Zanko (CRT),2023-2023,1,1,1.0,0,4,0,-,-,4.00,-,0,0,V Zanko,CRT


In [ ]:
df4 = df4.drop('Span', axis=1)
df4

,Player,Matches,Innings,Overs,Maidens,Runs,Wickets,BBI,Average,Economy,SR,Four_Wickets,Five_Wickets,Player_Name,Country
0,BJ McCarthy (IRE),21,21,75.4,1,605,22,3/7,27.50,7.99,20.6,0,0,BJ McCarthy,IRE
1,AS Joseph (WI),9,9,34.0,0,253,18,5/40,14.05,7.44,11.3,1,1,AS Joseph,WI
2,CA Young (IRE),8,8,30.0,0,210,18,4/28,11.66,7.00,10.0,1,0,CA Young,IRE
3,M Akayezu (RWN),13,13,40.0,2,297,17,3/27,17.47,7.42,14.1,0,0,M Akayezu,RWN
4,L Ngidi (SA),10,10,33.0,0,324,17,5/39,19.05,9.81,11.6,1,1,L Ngidi,SA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
581,A Yon (STHEL),1,1,1.0,0,11,0,-,-,11.00,-,0,0,A Yon,STHEL
582,Zabiullah Zahid (SWE),1,1,4.0,0,34,0,-,-,8.50,-,0,0,Zabiullah Zahid,SWE
583,Zain Ul Abidin (Saudi),1,1,4.0,0,22,0,-,-,5.50,-,0,0,Zain Ul Abidin,Saudi
584,V Zanko (CRT),1,1,1.0,0,4,0,-,-,4.00,-,0,0,V Zanko,CRT


In [ ]:
def replace_values(value):
    if value == 'DNB':
      return 0
    elif value == 'TDNB':
      return 0
    elif value == 'absent':
      return 0
    elif value == 'sub':
      return 0
    elif value == '-':
      return 0
    else:
      return value

# apply the replace_values function to all columns in the DataFrame
df4 = df4.applymap(replace_values)
df4

,Player,Matches,Innings,Overs,Maidens,Runs,Wickets,BBI,Average,Economy,SR,Four_Wickets,Five_Wickets,Player_Name,Country
0,BJ McCarthy (IRE),21,21,75.4,1,605,22,3/7,27.50,7.99,20.6,0,0,BJ McCarthy,IRE
1,AS Joseph (WI),9,9,34.0,0,253,18,5/40,14.05,7.44,11.3,1,1,AS Joseph,WI
2,CA Young (IRE),8,8,30.0,0,210,18,4/28,11.66,7.00,10.0,1,0,CA Young,IRE
3,M Akayezu (RWN),13,13,40.0,2,297,17,3/27,17.47,7.42,14.1,0,0,M Akayezu,RWN
4,L Ngidi (SA),10,10,33.0,0,324,17,5/39,19.05,9.81,11.6,1,1,L Ngidi,SA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
581,A Yon (STHEL),1,1,1.0,0,11,0,0,0,11.00,0,0,0,A Yon,STHEL
582,Zabiullah Zahid (SWE),1,1,4.0,0,34,0,0,0,8.50,0,0,0,Zabiullah Zahid,SWE
583,Zain Ul Abidin (Saudi),1,1,4.0,0,22,0,0,0,5.50,0,0,0,Zain Ul Abidin,Saudi
584,V Zanko (CRT),1,1,1.0,0,4,0,0,0,4.00,0,0,0,V Zanko,CRT


In [ ]:
df4.to_excel('First_change_bowlers.xlsx')

## Second Change

In [ ]:
# Define the initial page URL
url = 'https://stats.espncricinfo.com/ci/engine/stats/index.html?bowling_positionmax2=4;bowling_positionmin2=4;bowling_positionval2=bowling_position;class=3;filter=advanced;home_or_away=1;home_or_away=2;home_or_away=3;innings_number=1;innings_number=2;orderby=wickets;page=1;size=200;spanmin1=1+Jan+2022;spanval1=span;template=results;type=bowling'

response = requests.get(url)
response = response.content
soup = BeautifulSoup(response, 'html.parser')

In [ ]:
# Initialize the page number
page_number = 1
total_pages = int(soup.find_all('td', class_='left')[6].text.split(' ')[6].rstrip())

In [ ]:
page_number

1

In [ ]:
total_pages

3

In [ ]:
data = []
for i in range(1, total_pages+1):
  url = f"https://stats.espncricinfo.com/ci/engine/stats/index.html?bowling_positionmax2=4;bowling_positionmin2=4;bowling_positionval2=bowling_position;class=3;filter=advanced;home_or_away=1;home_or_away=2;home_or_away=3;innings_number=1;innings_number=2;orderby=wickets;page={i};size=200;spanmin1=1+Jan+2022;spanval1=span;template=results;type=bowling"
  response = requests.get(url)
  response = response.content
  soup = BeautifulSoup(response, 'html.parser')
  # Find the table element that contains the data we want to extract
  table = soup.select_one('#ciHomeContentlhs > div.pnl650M > table:nth-child(5)')

  # Check if the table exists
  if table is not None:
    # Loop through the rows of the table and extract the data for each player
    for row in table.tbody.find_all('tr'):
      columns = row.find_all('td')
      if len(columns) > 0:
        Player = columns[0].text.strip()
        Span = columns[1].text.strip()
        Matches = columns[2].text.strip()
        Innings = columns[3].text.strip()
        Overs = columns[4].text.strip()
        Maidens = columns[5].text.strip()
        Runs = columns[6].text.strip()
        Wickets = columns[7].text.strip()
        BBI = columns[8].text.strip()
        Average = columns[9].text.strip()
        Economy = columns[10].text.strip()
        SR = columns[11].text.strip()
        Four_Wickets = columns[12].text.strip()
        Five_Wickets = columns[13].text.strip()


        # Add the data for each player to the DataFrame
        data.append({
          'Player': Player,
          'Span': Span,
          'Matches': Matches,
          'Innings': Innings,
          'Overs' : Overs,
          'Maidens': Maidens,
          'Runs': Runs,
          'Wickets' : Wickets,
          'BBI': BBI,
          'Average': Average,
          'Economy': Economy,
          'SR': SR,
          'Four_Wickets' : Four_Wickets,
          'Five_Wickets': Five_Wickets
        })

    # Check if there is a next page by finding the 'Next' button on the page
    next_page_link = soup.select_one('.PaginationLink')
    if next_page_link is not None:
      # If there is a next page, increment the page number and update the URL to the next page
      i += 1
    else:
      # If there is no next page, break out of the loop
      break

# Create a DataFrame from the scraped data
df4 = pd.DataFrame(data)

In [ ]:
# Print the DataFrame
df4.head(10)

,Player,Span,Matches,Innings,Overs,Maidens,Runs,Wickets,BBI,Average,Economy,SR,Four_Wickets,Five_Wickets
0,D Maisuria (BOT),2022-2023,8,8,30.4,2,150,27,5/18,5.55,4.89,6.8,2,2
1,H Ssenyondo (UGA),2022-2023,13,13,47.4,3,239,20,4/7,11.95,5.01,14.3,2,0
2,Haris Rauf (PAK),2022-2023,13,13,47.3,1,312,19,4/18,16.42,6.56,15.0,2,0
3,SM Curran (ENG),2022-2022,8,8,30.4,0,215,17,5/10,12.64,7.01,10.8,0,1
4,YM Nkanya (TAN),2022-2023,8,8,29.0,1,147,16,4/10,9.18,5.06,10.8,1,0
5,Rashid Khan (AFG),2022-2023,13,13,50.0,0,343,16,3/22,21.43,6.86,18.7,0,0
6,PWH de Silva (SL),2022-2023,10,10,36.0,0,294,15,3/27,19.60,8.16,14.4,0,0
7,Vraj Patel (KENYA),2022-2023,8,8,32.0,1,156,15,3/12,10.40,4.87,12.8,0,0
8,YS Chahal (IND),2022-2023,11,11,39.2,0,351,13,3/20,27.00,8.92,18.1,0,0
9,BFW de Leede (NED),2022-2022,8,8,23.0,0,156,13,3/19,12.00,6.78,10.6,0,0


In [ ]:
# Extract player name and country using regular expression
df4[['Player_Name', 'Country']] = df4['Player'].str.extract(r'^(.*?) \((.*?)\)')

In [ ]:
df4

,Player,Span,Matches,Innings,Overs,Maidens,Runs,Wickets,BBI,Average,Economy,SR,Four_Wickets,Five_Wickets,Player_Name,Country
0,D Maisuria (BOT),2022-2023,8,8,30.4,2,150,27,5/18,5.55,4.89,6.8,2,2,D Maisuria,BOT
1,H Ssenyondo (UGA),2022-2023,13,13,47.4,3,239,20,4/7,11.95,5.01,14.3,2,0,H Ssenyondo,UGA
2,Haris Rauf (PAK),2022-2023,13,13,47.3,1,312,19,4/18,16.42,6.56,15.0,2,0,Haris Rauf,PAK
3,SM Curran (ENG),2022-2022,8,8,30.4,0,215,17,5/10,12.64,7.01,10.8,0,1,SM Curran,ENG
4,YM Nkanya (TAN),2022-2023,8,8,29.0,1,147,16,4/10,9.18,5.06,10.8,1,0,YM Nkanya,TAN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
578,M Yunusu Issa (TAN),2022-2022,1,1,2.0,0,12,0,-,-,6.00,-,0,0,M Yunusu Issa,TAN
579,Zahid Khan (SLE),2022-2022,1,1,1.0,0,7,0,-,-,7.00,-,0,0,Zahid Khan,SLE
580,Zain Ahmad (Fran),2023-2023,1,1,2.0,0,19,0,-,-,9.50,-,0,0,Zain Ahmad,Fran
581,A Zampa (AUS),2022-2022,1,1,2.0,0,17,0,-,-,8.50,-,0,0,A Zampa,AUS


In [ ]:
df4 = df4.drop('Span', axis=1)
df4

,Player,Matches,Innings,Overs,Maidens,Runs,Wickets,BBI,Average,Economy,SR,Four_Wickets,Five_Wickets,Player_Name,Country
0,D Maisuria (BOT),8,8,30.4,2,150,27,5/18,5.55,4.89,6.8,2,2,D Maisuria,BOT
1,H Ssenyondo (UGA),13,13,47.4,3,239,20,4/7,11.95,5.01,14.3,2,0,H Ssenyondo,UGA
2,Haris Rauf (PAK),13,13,47.3,1,312,19,4/18,16.42,6.56,15.0,2,0,Haris Rauf,PAK
3,SM Curran (ENG),8,8,30.4,0,215,17,5/10,12.64,7.01,10.8,0,1,SM Curran,ENG
4,YM Nkanya (TAN),8,8,29.0,1,147,16,4/10,9.18,5.06,10.8,1,0,YM Nkanya,TAN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
578,M Yunusu Issa (TAN),1,1,2.0,0,12,0,-,-,6.00,-,0,0,M Yunusu Issa,TAN
579,Zahid Khan (SLE),1,1,1.0,0,7,0,-,-,7.00,-,0,0,Zahid Khan,SLE
580,Zain Ahmad (Fran),1,1,2.0,0,19,0,-,-,9.50,-,0,0,Zain Ahmad,Fran
581,A Zampa (AUS),1,1,2.0,0,17,0,-,-,8.50,-,0,0,A Zampa,AUS


In [ ]:
def replace_values(value):
    if value == 'DNB':
      return 0
    elif value == 'TDNB':
      return 0
    elif value == 'absent':
      return 0
    elif value == 'sub':
      return 0
    elif value == '-':
      return 0
    else:
      return value

# apply the replace_values function to all columns in the DataFrame
df4 = df4.applymap(replace_values)
df4

,Player,Matches,Innings,Overs,Maidens,Runs,Wickets,BBI,Average,Economy,SR,Four_Wickets,Five_Wickets,Player_Name,Country
0,D Maisuria (BOT),8,8,30.4,2,150,27,5/18,5.55,4.89,6.8,2,2,D Maisuria,BOT
1,H Ssenyondo (UGA),13,13,47.4,3,239,20,4/7,11.95,5.01,14.3,2,0,H Ssenyondo,UGA
2,Haris Rauf (PAK),13,13,47.3,1,312,19,4/18,16.42,6.56,15.0,2,0,Haris Rauf,PAK
3,SM Curran (ENG),8,8,30.4,0,215,17,5/10,12.64,7.01,10.8,0,1,SM Curran,ENG
4,YM Nkanya (TAN),8,8,29.0,1,147,16,4/10,9.18,5.06,10.8,1,0,YM Nkanya,TAN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
578,M Yunusu Issa (TAN),1,1,2.0,0,12,0,0,0,6.00,0,0,0,M Yunusu Issa,TAN
579,Zahid Khan (SLE),1,1,1.0,0,7,0,0,0,7.00,0,0,0,Zahid Khan,SLE
580,Zain Ahmad (Fran),1,1,2.0,0,19,0,0,0,9.50,0,0,0,Zain Ahmad,Fran
581,A Zampa (AUS),1,1,2.0,0,17,0,0,0,8.50,0,0,0,A Zampa,AUS


In [ ]:
df4.to_excel('Second_change_bowlers.xlsx')

## Others

In [ ]:
# Define the initial page URL
url = 'https://stats.espncricinfo.com/ci/engine/stats/index.html?bowling_positionmax2=11;bowling_positionmin2=5;bowling_positionval2=bowling_position;class=3;filter=advanced;home_or_away=1;home_or_away=2;home_or_away=3;innings_number=1;innings_number=2;orderby=wickets;page=1;size=200;spanmin1=1+Jan+2022;spanval1=span;template=results;type=bowling'

response = requests.get(url)
response = response.content
soup = BeautifulSoup(response, 'html.parser')

In [ ]:
# Initialize the page number
page_number = 1
total_pages = int(soup.find_all('td', class_='left')[6].text.split(' ')[6].rstrip())

In [ ]:
page_number

1

In [ ]:
total_pages

5

In [ ]:
data = []
for i in range(1, total_pages+1):
  url = f"https://stats.espncricinfo.com/ci/engine/stats/index.html?bowling_positionmax2=11;bowling_positionmin2=5;bowling_positionval2=bowling_position;class=3;filter=advanced;home_or_away=1;home_or_away=2;home_or_away=3;innings_number=1;innings_number=2;orderby=wickets;page={i};size=200;spanmin1=1+Jan+2022;spanval1=span;template=results;type=bowling"
  response = requests.get(url)
  response = response.content
  soup = BeautifulSoup(response, 'html.parser')
  # Find the table element that contains the data we want to extract
  table = soup.select_one('#ciHomeContentlhs > div.pnl650M > table:nth-child(5)')

  # Check if the table exists
  if table is not None:
    # Loop through the rows of the table and extract the data for each player
    for row in table.tbody.find_all('tr'):
      columns = row.find_all('td')
      if len(columns) > 0:
        Player = columns[0].text.strip()
        Span = columns[1].text.strip()
        Matches = columns[2].text.strip()
        Innings = columns[3].text.strip()
        Overs = columns[4].text.strip()
        Maidens = columns[5].text.strip()
        Runs = columns[6].text.strip()
        Wickets = columns[7].text.strip()
        BBI = columns[8].text.strip()
        Average = columns[9].text.strip()
        Economy = columns[10].text.strip()
        SR = columns[11].text.strip()
        Four_Wickets = columns[12].text.strip()
        Five_Wickets = columns[13].text.strip()


        # Add the data for each player to the DataFrame
        data.append({
          'Player': Player,
          'Span': Span,
          'Matches': Matches,
          'Innings': Innings,
          'Overs' : Overs,
          'Maidens': Maidens,
          'Runs': Runs,
          'Wickets' : Wickets,
          'BBI': BBI,
          'Average': Average,
          'Economy': Economy,
          'SR': SR,
          'Four_Wickets' : Four_Wickets,
          'Five_Wickets': Five_Wickets
        })

    # Check if there is a next page by finding the 'Next' button on the page
    next_page_link = soup.select_one('.PaginationLink')
    if next_page_link is not None:
      # If there is a next page, increment the page number and update the URL to the next page
      i += 1
    else:
      # If there is no next page, break out of the loop
      break

# Create a DataFrame from the scraped data
df4 = pd.DataFrame(data)

In [ ]:
# Print the DataFrame
df4.head(10)

,Player,Span,Matches,Innings,Overs,Maidens,Runs,Wickets,BBI,Average,Economy,SR,Four_Wickets,Five_Wickets
0,IS Sodhi (NZ),2022-2023,27,27,100.4,0,803,29,4/28,27.68,7.97,20.8,1,0
1,S Lamichhane (NEP),2022-2022,11,11,43.0,0,250,24,3/9,10.41,5.81,10.7,0,0
2,Shadab Khan (PAK),2022-2023,17,17,58.4,0,430,24,4/8,17.91,7.32,14.6,1,0
3,K Nassoro (TAN),2022-2023,23,23,71.0,1,451,23,4/14,19.60,6.35,18.5,1,0
4,PWH de Silva (SL),2022-2023,14,14,54.0,1,398,20,3/8,19.90,7.37,16.2,0,0
5,Sikandar Raza (ZIM),2022-2022,17,17,53.0,1,296,20,4/8,14.80,5.58,15.9,1,0
6,SVE Bharathi (GER),2022-2023,16,16,49.4,1,312,19,4/6,16.42,6.28,15.6,1,0
7,K Irakoze (RWN),2022-2023,17,17,54.4,0,374,18,3/15,20.77,6.84,18.2,0,0
8,YM Nkanya (TAN),2022-2022,14,14,47.0,1,213,18,4/1,11.83,4.53,15.6,1,0
9,OF Smith (WI),2022-2023,14,14,38.0,0,354,17,3/29,20.82,9.31,13.4,0,0


In [ ]:
# Extract player name and country using regular expression
df4[['Player_Name', 'Country']] = df4['Player'].str.extract(r'^(.*?) \((.*?)\)')

In [ ]:
df4

,Player,Span,Matches,Innings,Overs,Maidens,Runs,Wickets,BBI,Average,Economy,SR,Four_Wickets,Five_Wickets,Player_Name,Country
0,IS Sodhi (NZ),2022-2023,27,27,100.4,0,803,29,4/28,27.68,7.97,20.8,1,0,IS Sodhi,NZ
1,S Lamichhane (NEP),2022-2022,11,11,43.0,0,250,24,3/9,10.41,5.81,10.7,0,0,S Lamichhane,NEP
2,Shadab Khan (PAK),2022-2023,17,17,58.4,0,430,24,4/8,17.91,7.32,14.6,1,0,Shadab Khan,PAK
3,K Nassoro (TAN),2022-2023,23,23,71.0,1,451,23,4/14,19.60,6.35,18.5,1,0,K Nassoro,TAN
4,PWH de Silva (SL),2022-2023,14,14,54.0,1,398,20,3/8,19.90,7.37,16.2,0,0,PWH de Silva,SL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
881,BAA Zammitt (GIBR),2023-2023,1,1,0.5,0,5,0,-,-,6.00,-,0,0,BAA Zammitt,GIBR
882,V Zanko (CRT),2023-2023,1,1,2.0,0,25,0,-,-,12.50,-,0,0,V Zanko,CRT
883,Ziaur Rehman (Fin),2022-2022,1,1,1.0,0,11,0,-,-,11.00,-,0,0,Ziaur Rehman,Fin
884,N Zimonjic (SRB),2022-2022,1,1,1.0,0,12,0,-,-,12.00,-,0,0,N Zimonjic,SRB


In [ ]:
df4 = df4.drop('Span', axis=1)
df4

,Player,Matches,Innings,Overs,Maidens,Runs,Wickets,BBI,Average,Economy,SR,Four_Wickets,Five_Wickets,Player_Name,Country
0,IS Sodhi (NZ),27,27,100.4,0,803,29,4/28,27.68,7.97,20.8,1,0,IS Sodhi,NZ
1,S Lamichhane (NEP),11,11,43.0,0,250,24,3/9,10.41,5.81,10.7,0,0,S Lamichhane,NEP
2,Shadab Khan (PAK),17,17,58.4,0,430,24,4/8,17.91,7.32,14.6,1,0,Shadab Khan,PAK
3,K Nassoro (TAN),23,23,71.0,1,451,23,4/14,19.60,6.35,18.5,1,0,K Nassoro,TAN
4,PWH de Silva (SL),14,14,54.0,1,398,20,3/8,19.90,7.37,16.2,0,0,PWH de Silva,SL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
881,BAA Zammitt (GIBR),1,1,0.5,0,5,0,-,-,6.00,-,0,0,BAA Zammitt,GIBR
882,V Zanko (CRT),1,1,2.0,0,25,0,-,-,12.50,-,0,0,V Zanko,CRT
883,Ziaur Rehman (Fin),1,1,1.0,0,11,0,-,-,11.00,-,0,0,Ziaur Rehman,Fin
884,N Zimonjic (SRB),1,1,1.0,0,12,0,-,-,12.00,-,0,0,N Zimonjic,SRB


In [ ]:
def replace_values(value):
    if value == 'DNB':
      return 0
    elif value == 'TDNB':
      return 0
    elif value == 'absent':
      return 0
    elif value == 'sub':
      return 0
    elif value == '-':
      return 0
    else:
      return value

# apply the replace_values function to all columns in the DataFrame
df4 = df4.applymap(replace_values)
df4

,Player,Matches,Innings,Overs,Maidens,Runs,Wickets,BBI,Average,Economy,SR,Four_Wickets,Five_Wickets,Player_Name,Country
0,IS Sodhi (NZ),27,27,100.4,0,803,29,4/28,27.68,7.97,20.8,1,0,IS Sodhi,NZ
1,S Lamichhane (NEP),11,11,43.0,0,250,24,3/9,10.41,5.81,10.7,0,0,S Lamichhane,NEP
2,Shadab Khan (PAK),17,17,58.4,0,430,24,4/8,17.91,7.32,14.6,1,0,Shadab Khan,PAK
3,K Nassoro (TAN),23,23,71.0,1,451,23,4/14,19.60,6.35,18.5,1,0,K Nassoro,TAN
4,PWH de Silva (SL),14,14,54.0,1,398,20,3/8,19.90,7.37,16.2,0,0,PWH de Silva,SL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
881,BAA Zammitt (GIBR),1,1,0.5,0,5,0,0,0,6.00,0,0,0,BAA Zammitt,GIBR
882,V Zanko (CRT),1,1,2.0,0,25,0,0,0,12.50,0,0,0,V Zanko,CRT
883,Ziaur Rehman (Fin),1,1,1.0,0,11,0,0,0,11.00,0,0,0,Ziaur Rehman,Fin
884,N Zimonjic (SRB),1,1,1.0,0,12,0,0,0,12.00,0,0,0,N Zimonjic,SRB


In [ ]:
df4.to_excel('Other_changes_bowlers.xlsx')